In [307]:
import pandas as pd
import numpy as np

from tqdm import tqdm as tq

In [381]:
import symspellpy

In [387]:
!opencorpora download

Creating annot.opcorpora.xml from http://opencorpora.org/files/export/annot/annot.opcorpora.xml.bz2
.............................................................................................................................
Done.


In [388]:
import opencorpora

corpus = opencorpora.CorpusReader('annot.opcorpora.xml')
catalog = corpus.catalog()

In [390]:
def recursive_flatten_generator(array):
    lst = []
    for i in array:
        if isinstance(i, list):
            lst.extend(recursive_flatten_generator(i))
        else:
            lst.append(i)
    return lst

def read_list(filepath, codec):
    with codecs.open(filepath, 'r',codec) as file:
        data = file.readlines()
        data = [line.rstrip() for line in data]
    return data

In [392]:

from  more_itertools import unique_everseen

In [ ]:
corp_index = [item[0] for item in catalog]
texts = []
for ind in tq(corp_index):
    corp = corpus.words(ind)
    texts.append(corp)

In [393]:
texts = list(unique_everseen(texts))
#len(texts)
texts = recursive_flatten_generator(texts)

In [394]:
sym_spell = symspellpy.SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
sym_spell.create_dictionary(texts, encoding='utf-8')

True

In [308]:
df_anamnesises = pd.read_csv('data/anamnesises_with_norm_diagnosis.csv', sep='\t')

In [309]:
anamnesises = df_anamnesises.anamnesise_without_tags
anamnesises.dropna(inplace=True)

In [310]:
import re

In [344]:
anamnesises = anamnesises.str.replace('_', ' ')
anamnesises = anamnesises.str.replace(':', ': ')
anamnesises = anamnesises.str.replace(',', ', ')
anamnesises = anamnesises.str.replace('.', '. ') #отдельно обработать аббревиатуры типа 'ф.кл.'

In [345]:
#'Diva004' -> 'Diva 004'- divides letters and digits
R = r'(?i)(?:(?<=\d)(?=[а-яa-zё])|(?<=[а-яa-zё])(?=\d))'
sep_tags = [re.sub(R, ' ', item) for item in tq(anamnesises)]

#'какZUMBA' -> 'как ZUMBA' - divides cases
R_1 = r'(?:(?<=[a-zа-яё])(?=[А-ЯA-ZЁ]))'
sep_tags = [re.sub(R_1, ' ', item) for item in tq(sep_tags)]

#'душноvsдует' -> 'душно vs дует' - divides languages (en, ru)
R_2 = r'(?i)(?:(?<=[a-z])(?=[а-яё])|(?<=[а-яё])(?=[a-z]))'
sep_tags = [re.sub(R_2, ' ', item) for item in tq(sep_tags)]

#'ДушноДует' -> 'Душно Дует' - divides camel case
R_3 = r'(?:(?<=[a-zа-яё])(?=[A-ZА-ЯЁ]))'
sep_tags = [re.sub(R_3, ' ', item) for item in tq(sep_tags)]

#'вышел.Он' -> 'вышел. Он' - divides Upper case after dot

R_4 = r'(?:(?<=[.])(?=[A-Z|А-Я]))'
anamnesises = pd.Series([re.sub(R_4, ' ', item) for item in tq(sep_tags)])

100%|██████████| 140746/140746 [00:00<00:00, 371410.59it/s]


In [ ]:
div_tags = [sym_spell.word_segmentation(item, ignore_token=r'\d+', max_edit_distance=1).corrected_string for item in tq(anamnesises)]

  4%|▍         | 5378/140746 [12:11<6:31:11,  5.77it/s] 

In [346]:
from collections import Counter, defaultdict

Аббревиатуры бывают:
- Обычно записаны слитно
- Записаны upper-case
- Заканчиваются на .

In [347]:
#upper case аббревиатуры

upper_abbr_counter = Counter()

for text in anamnesises:
    for word in text.split():
        if word.isupper():
            upper_abbr_counter[word] += 1

In [348]:
upper_abbr_counter.most_common(10)

[('ГБ', 12977),
 ('II', 9711),
 ('ИБС', 6010),
 ('ОСТ', 2960),
 ('ЖКБ', 2916),
 ('ОНМК', 2582),
 ('III', 2461),
 ('В', 2397),
 ('МКБ', 2380),
 ('ГБ,', 2006)]

In [349]:
#заканчиваются на точку

ends_dot_abbr_counter = Counter()

for text in anamnesises:
    for word in text.split():
        if '.' in word and not word.isupper():
            ends_dot_abbr_counter[word] += 1

In [351]:
ends_dot_abbr_counter.most_common(10)

[('г.', 7441),
 ('ст.', 3791),
 ('.', 3391),
 ('ремиссия.', 1780),
 ('отрицает.', 1467),
 ('лет.', 1307),
 ('4.', 1167),
 ('04.', 1071),
 ('обострения.', 1031),
 ('конечностей.', 984)]

In [352]:
dot_abbrs = list(ends_dot_abbr_counter.keys())

In [353]:
dot_abbrs_len = list(map(len, dot_abbrs))

In [356]:
df_dot_abbrs = pd.DataFrame(np.vstack((dot_abbrs, dot_abbrs_len)).T, columns=['abbr', 'length'])

In [357]:
df_dot_abbrs

,abbr,length
0,поясничный.,11
1,ноге.,5
2,редко.,6
3,Знакома.,8
4,отр.,4
...,...,...
6606,колпроктэктомия.,16
6607,почку.,6
6608,почка).,7
6609,центра.,7


In [361]:
df_dot_abbrs.length = df_dot_abbrs.length.astype('int')

In [362]:
len_perc_25, len_perc_75 = np.percentile(df_dot_abbrs.length, [0, 60])
high_len = len_perc_75 + 1.5 * (len_perc_75 - len_perc_25)

In [363]:
high_len

23.5

In [367]:
df_dot_abbrs.query('length < %d' % high_len).head()

,abbr,length
0,поясничный.,11
1,ноге.,5
2,редко.,6
3,Знакома.,8
4,отр.,4


In [376]:
df_dot_abbrs.length.sort_values()

65       1
84       2
4747     2
5487     2
1035     2
        ..
179     30
1455    31
3130    33
4637    33
1410    35
Name: length, Length: 6611, dtype: int64

In [377]:
df_dot_abbrs.query('length == %d' % 33).head()

,abbr,length
3130,постинфекционный+неврозоподобный.,33
4637,Атеросклеротическийкардиосклероз.,33


In [378]:
df_dot_abbrs.query('length == %d' % max(df_dot_abbrs.length)).head()

,abbr,length
1410,Работаетадминистратором-менеджером.,35


In [327]:
ends_dot_abbr_counter.most_common(10)

[('г.', 6358),
 ('.', 3027),
 ('ст.', 2832),
 ('ремиссия.', 1745),
 ('отрицает.', 1449),
 ('лет.', 1278),
 ('4.', 1112),
 ('обострения.', 1011),
 ('конечностей.', 968),
 ('нет.', 731)]

In [328]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [57]:
for encode, _ in upper_abbr_counter.most_common(5):
    print('Аббревиатура', encode)
    
    for ind, text in enumerate(anamnesises[anamnesises_splitted.apply(lambda x: encode in x)][:3]):
        text = text.replace(encode, bcolors.WARNING + encode + bcolors.ENDC)
        
        print('текст #%d' % (ind + 1))
        print(text, '\n')

Аббревиатура ГБ
текст #1
 Хронические заболевания в анамнезе  : ГБ ; ИБС : с нарушением ритма ; остеохондроз позвоночника ; облитерирующий атеросклероз артерий нижних конечностей ; хронический пиелонефрит 

текст #2
 Хронические заболевания в анамнезе  : ГБ ; ИБС ; стенокардия напряжения, коронарография от 2008,2011 ; Аденомы надпочечников несекретирующие 

текст #3
 Хронические заболевания в анамнезе  : ГБ ; ИБС ; стенокардия ; остеохондроз позвоночника ; варикозная болезнь вен нижних конечностей ; аутоиммунный тиреоидит 

Аббревиатура II
текст #1
ИБС. Атеросклеротический кардиосклероз. Гипертоническая болезнь III ст. Риск 4. Имплантация ПЭКС DDD по поводу бинодальной слабости (субтотальная АВ-блокада, СССУ) в 2009 г. Пароксизм фибрилляции-трепетания предсердий 10.10.2013 г. ХСН II ФК. Дислипидемия. ЖКБ. Хр.холецистит, ремиссия. Мастэктомия слева в 2003 г. по поводу Са. ДУЗ. Эутиреоз. 

текст #2
 Хронические заболевания в анамнезе  : Гипертоническая болезнь III ст. Риск 4. ИБС. Стенок